In [21]:
import pandas as pd
from pathlib import Path
import folium
from sklearn.cluster import DBSCAN
import numpy as np

In [22]:
HEATMAP_CSV = Path('heatmap.csv')

In [23]:
hm = pd.read_csv(HEATMAP_CSV)
hm

,latitude,longitude,intensity
0,33.697423,-117.813692,0.154382
1,33.697423,-117.813649,0.203886
2,33.697423,-117.813606,0.159789
3,33.697387,-117.813735,0.244870
4,33.697387,-117.813692,0.431655
...,...,...,...
11896,33.665997,-117.826695,0.110651
11897,33.665997,-117.826653,0.237577
11898,33.665997,-117.826610,0.310309
11899,33.665997,-117.826567,0.308576


In [24]:
hm_f = hm[hm.intensity > 0.70]

In [25]:
irvine_map = folium.Map(
    zoom_start=12,
    location=[33.669445, -117.823059]
)

for i, r in hm_f.iterrows():
    #setting for the popup
    #Plotting the Marker for each stationsト
    folium.map.Marker(
        location=[r['latitude'], r['longitude']], 
        icon=folium.Icon(color="green",icon="person-running", prefix='fa')
    ).add_to(irvine_map)
    
    folium.vector_layers.CircleMarker(
        location=[r['latitude'], r['longitude']], 
        radius=0.1,
        color='#3186cc',
        fill_color='#3186cc'
    ).add_to(irvine_map)

irvine_map

In [26]:
db = DBSCAN(eps=0.1/6371., algorithm='ball_tree', metric='haversine').fit(np.radians(hm_f[['latitude', 'longitude']]))
hm_fl = hm_f.assign(label=pd.Series(db.labels_).values)
hm_fl

,latitude,longitude,intensity,label
139,33.696780,-117.814894,0.734019,0
140,33.696780,-117.814851,0.905903,0
141,33.696780,-117.814808,0.868720,0
142,33.696780,-117.814765,0.713819,0
155,33.696744,-117.814894,0.823350,0
...,...,...,...,...
11745,33.666461,-117.827210,0.709249,15
11757,33.666425,-117.827168,0.769526,15
11884,33.666068,-117.826653,0.706367,15
11885,33.666068,-117.826610,0.777553,15


In [27]:
labels = hm_fl.label.unique()
clusters = {label : pd.DataFrame() for label in labels}

for key in clusters.keys():
    clusters[key] = hm_fl[:][hm_fl.label == key]

In [28]:
cluster_len = []

for idx, cluster in sorted(clusters.items()):
    cluster_len.append((len(cluster), idx))

cluster_len = sorted(cluster_len, reverse=True)
cluster_len

[(90, 14),
 (46, 12),
 (43, 0),
 (40, 13),
 (35, 10),
 (31, 11),
 (30, 5),
 (24, 3),
 (24, -1),
 (22, 4),
 (19, 9),
 (12, 8),
 (11, 1),
 (9, 2),
 (6, 7),
 (6, 6),
 (5, 15)]

In [29]:
dense_labels = [label for _, label in cluster_len[:len(clusters)//4]]
hm_dense = hm_fl[hm_fl.label.isin(dense_labels)]

In [30]:
irvine_dense_map = folium.Map(
    zoom_start=12,
    location=[33.669445, -117.823059]
)

for i, r in hm_dense.iterrows():
    #setting for the popup
    #Plotting the Marker for each stationsト
    folium.map.Marker(
        location=[r['latitude'], r['longitude']], 
        icon=folium.Icon(color="green",icon="person-running", prefix='fa')
    ).add_to(irvine_dense_map)
    
    folium.vector_layers.CircleMarker(
        location=[r['latitude'], r['longitude']], 
        radius=0.1,
        color='#3186cc',
        fill_color='#3186cc'
    ).add_to(irvine_dense_map)

irvine_dense_map